# NRC Emotions lexicon based text and sentiment analysis between two text corpuses
This analyses the two corpuses PA and YT on the basis of association of words with eight emotions (anger, fear, anticipation, trust, surprise, sadness, joy, and disgust) and two sentiments (negative and positive). The [NRC lexicon](https://saifmohammad.com/WebPages/NRC-Emotion-Lexicon.htm) is used.


# Load the python packages and initialize global variables

In [1]:
import docx
import gensim
import os
import pandas as pd
import nltk
import numpy as np
import re
import spacy
import sys
from scipy.stats import mannwhitneyu
from nltk.stem import WordNetLemmatizer
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

import pyLDAvis
import pyLDAvis.sklearn
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

nlp = spacy.load('en', disable=['parser', 'ner'])
sys.getdefaultencoding()

NRC_EMOTION_LEXICON_PATH = "Input/NRC-Emotion-Lexicon/NRC-Emotion-Lexicon-v0.92/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt"

verbs = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
nouns = ['NNS', 'NNPS']  # + 'NN' + 'NNP',
adjectives = ['JJ', 'JJR', 'JJS']

OUTPUT_PATH = "Output"
PA_CORPUS_TEXT = "PACorpus.txt"
YT_CORPUS_TEXT = "YTCorpus.txt"
CORPUS_COMPARISON_FILENAME = "CorpusCompare.csv"
INPUT_PATH = "Input"
INPUT_FILE = "US3_ALL_TRANSCRIPTS.docx"
WORD_LIST = ['i', 'me', 'our', 'my', 'we', 'us', 'you', 'your', 'she', 'her', 'he', 'him', 'his', 'they','them', 'their']

# Read given input document file 
Read the given input file and return list of lines

In [2]:
def read_input_file(file_path):
    doc = docx.Document(file_path)
    total_lines = list()
    for i in doc.paragraphs:
        total_lines.append(i.text)
    return total_lines
filename = 'Input/US3_ALL_TRANSCRIPTS.docx'
lines = read_input_file(filename)

# Grouping of Corpuses
This method groups the given text into two groups (PA and YT) and removes unnecessary lines and characters.

In [3]:
def group_to_corpuses(lines_in_input):
    yt_group = list()
    pa_group = list()
    initial_group_flag = None
    regex = re.compile('^P[0-9]+$')

    for line in lines_in_input:
        if not line.strip():
            continue
        if line.strip().startswith("Joni: ") or line.strip().startswith("Jim:"):
            continue
        if line.strip().startswith("R:") or line.strip().startswith("R: ") or line.strip().startswith("R :"):
            continue
        if "R:" in line:
            lines_list = line.splitlines()
            for single_line in lines_list:
                if single_line.startswith("R:"):
                    lines_list.remove(single_line)
            line = ",".join(lines_list)

        if re.match(regex, line):
            continue
        if line.strip().startswith('2018-11-') or line.strip().startswith('Total experiment talk time:'):
            continue
        line = re.sub(r'\[[^()]*\]', '', line)  # regex removing text between brackets

        line = line.replace('...', ' ,')
        line = line.replace('…', ' ,')

        if not line.strip().startswith("P:") and not line.strip().startswith('YT') and not line.strip().startswith(
                'PA'):
            line = "".join([i if ord(i) < 128 else ' ' for i in line])
            if line.strip().startswith("R :") or line.strip().startswith("R4.") or line.strip().startswith("P10"):
                continue

        if line.strip().lower() == 'yt':
            initial_group_flag = 'yt'
            continue
        elif line.strip().lower() == 'pa':
            initial_group_flag = 'pa'
            continue
        elif line:
            if initial_group_flag == 'pa':
                pa_group.append(line)
            else:
                yt_group.append(line)
    return pa_group, yt_group
pa_group, yt_group = group_to_corpuses(lines)

# Tokenize and Clean-up
Tokenize each sentence into a list of words, removing punctuations and unnecessary characters altogether. Gensim’s ```simple_preprocess()``` is great for this. Additionally I have set ```deacc=True``` to remove the punctuations.

In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield (gensim.utils.simple_preprocess(str(sentence), deacc=True))
pa_sentences_group = sent_to_words(pa_group)
yt_sentences_group = sent_to_words(yt_group)

# Lemmatization
Lemmatization convert words to its root word. This method lemmatizes the tokenized words and joins them into sentences.

In [5]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append(" ".join(
            [token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out
pa_sentences_group = lemmatization(pa_sentences_group)
pa_sentences_group = ". ".join(pa_sentences_group)
yt_sentences_group = lemmatization(yt_sentences_group)
yt_sentences_group = ". ".join(yt_sentences_group)

# Splitting
Splitter class use english pickle splitter to split joined sentences before tokenization

In [6]:
class Splitter(object):
    def __init__(self):
        self.nltk_splitter = nltk.data.load('tokenizers/punkt/english.pickle')
        self.nltk_tokenizer = nltk.tokenize.TreebankWordTokenizer()

    def split(self, text):
        """
        input format: a paragraph of text
        output format: a list of lists of words.
            e.g.: [['this', 'is', 'a', 'sentence'], ['this', 'is', 'another', 'one']]
        """
        sentences = self.nltk_splitter.tokenize(text)
        tokenized_sentences = [self.nltk_tokenizer.tokenize(sent) for sent in sentences]
        return tokenized_sentences

# POS Tagging
Parts of Speech tagging of splitted words in a sentence.

In [7]:
class POSTagger(object):
    def __init__(self):
        pass

    def pos_tag(self, sentences):
        """
        input format: list of lists of words
            e.g.: [['this', 'is', 'a', 'sentence'], ['this', 'is', 'another', 'one']]
        output format: list of lists of tagged tokens. Each tagged tokens has a
        form, a lemma, and a list of tags
            e.g: [[('this', 'this', ['DT']), ('is', 'be', ['VB']), ('a', 'a', ['DT']), ('sentence', 'sentence', ['NN'])],
                    [('this', 'this', ['DT']), ('is', 'be', ['VB']), ('another', 'another', ['DT']), ('one', 'one', ['CARD'])]]
        """

        pos = [nltk.pos_tag(sentence) for sentence in sentences]
        # adapt format
        pos = [[(word, word, [postag]) for (word, postag) in sentence] for sentence in pos]
        return pos

# Dictionary Tagging of NRC emotion lexicon 
Read the NRC emotion lexicon and tag the words with emotion and lexicon. Use this tagged data to find the emotions and sentiment of each word of the POS tagged PA and YT corpus sentences.

In [8]:
class DictionaryTagger(object):
    def __init__(self, file_path):
        self.dictionary = dict()
        self.max_key_size = 0
        with open(file_path, "r", encoding="utf-8") as nrc_file:
            for line in nrc_file.readlines():
                if not line.strip():
                    continue
                line = re.sub(r'\s+', '\t', line)
                splited = line.replace("\n", "").split("\t")
                word, emotion, value = splited[0], splited[1], splited[2]

                if word in self.dictionary.keys():
                    self.dictionary[word].append((emotion, int(value)))
                else:
                    self.dictionary[word] = [(emotion, int(value))]

    def tag(self, postagged_sentences):
        return [self.tag_sentence(sentence) for sentence in postagged_sentences]

    def tag_sentence(self, sentence, tag_with_lemmas=False):
        """
        the result is only one tagging of all the possible ones.
        The resulting tagging is determined by these two priority rules:
            - longest matches have higher priority
            - search is made from left to right
        """
        tag_sentence = []
        N = len(sentence)
        if self.max_key_size == 0:
            self.max_key_size = N
        i = 0
        while i < N:
            j = min(i + self.max_key_size, N)  # avoid overflow
            tagged = False
            while j > i:
                expression_form = ' '.join([word[0] for word in sentence[i:j]]).lower()
                expression_lemma = ' '.join([word[1] for word in sentence[i:j]]).lower()
                if tag_with_lemmas:
                    literal = expression_lemma
                else:
                    literal = expression_form
                if literal in self.dictionary:
                    # self.logger.debug("found: %s" % literal)
                    is_single_token = j - i == 1
                    original_position = i
                    i = j
                    taggings = [tag for tag in self.dictionary[literal]]
                    tagged_expression = (expression_form, expression_lemma, taggings)
                    if is_single_token:  # if the tagged literal is a single token, conserve its previous taggings:
                        original_token_tagging = sentence[original_position][2]
                        tagged_expression[2].extend(original_token_tagging)
                    tag_sentence.append(tagged_expression)
                    tagged = True
                else:
                    j = j - 1
            if not tagged:
                tag_sentence.append(sentence[i])
                i += 1
        return tag_sentence

# MannWhitney-U test
Perform MannWhitney-U test to compare the emotions and sentiments between PA corpus and YT corpus

In [9]:
def mann_whitney_u_test(group_pa, group_yt):
    print("Mann Whitney-u Test:")
    pa_count = list(group_pa.values())
    yt_count = list(group_yt.values())
    try:
        mw_stat, mw_p = mannwhitneyu(pa_count, yt_count, use_continuity=True, alternative="two-sided")
    except ValueError:
        mw_stat = -1  # in case of ties, Mann-Whitney cannot rank, and so cannot calculate U
        mw_p = -1

    return mw_stat, mw_p

# Sentiment and Emotion Score Calculation
Get sentiments and emotions for each word from dictionary tagged data.

In [10]:
def get_sentiment(sentiment):
    if not isinstance(sentiment, tuple):
        return dict()
    results = dict()
    results.update({sentiment[0]: sentiment[1]})
    return results

Aggregate the sentiment score and emotions

In [11]:
def sentiment_score(dict_tagged_sentences):
    emotions = dict()
    for sentence in dict_tagged_sentences:
        for token in sentence:
            for tag in token[2]:
                value = get_sentiment(tag)
                if not value:
                    continue
                emotions.update({tag[0]: emotions.get(tag[0], 0) + value.get(tag[0], 0)})
    return emotions

Split the sentences in each corpus using the ```Splitter()``` we defined and POS tag the words using our ```POSTagger()``` before tagging these wors with sentiments and emotions using ```DictionaryTagger()```.

In [12]:
# read NRC emotion lexicon and tag these words with the given emotions and sentiments
dicttagger = DictionaryTagger(NRC_EMOTION_LEXICON_PATH)
# initialize 
postagger = POSTagger()
splitter = Splitter()

# split the pa corpus sentences using the splitter
splitted_pa_sentences = splitter.split(pa_sentences_group)

# POS tagging of splitted words in sentences
pos_tagged_pa_sentences = postagger.pos_tag(splitted_pa_sentences)

# emotions and sentiment tagging of the words in PA corpus using the loaded data from NRC emmotion lexicon
dict_tagged_pa_sentences = dicttagger.tag(pos_tagged_pa_sentences)

In [13]:
# split the yt corpus sentences using the splitter
splitted_yt_sentences = splitter.split(yt_sentences_group)

# POS tagging of splitted words in sentences
pos_tagged_yt_sentences = postagger.pos_tag(splitted_yt_sentences)

# emotions and sentiment tagging of the words in PA corpus using the loaded data from NRC emmotion lexicon
dict_tagged_yt_sentences = dicttagger.tag(pos_tagged_yt_sentences)

In [14]:
# calculate the emotions and sentiment score for PA corpus
pa_sentiment = sentiment_score(dict_tagged_pa_sentences)

# calculate the emotions and sentiment score for YT corpus
yt_sentiment = sentiment_score(dict_tagged_yt_sentences)

print("PA sentiment: %s" % pa_sentiment)
print("\nYT sentiment: %s\n" % yt_sentiment)
print("Is PA's Postive Sentiment > YT's Postive Sentiment? %s\n" % (pa_sentiment.get('positive') > yt_sentiment.get('positive')))

# calculate the MannWhitney-U test
mw_stat, mw_p = mann_whitney_u_test(pa_sentiment, yt_sentiment)
print("MannWhitney U Value: %s" % mw_stat)
print("MannWhitney rho Value: %s" % mw_p)

PA sentiment: {'anger': 57, 'anticipation': 353, 'disgust': 202, 'fear': 153, 'joy': 211, 'negative': 167, 'positive': 940, 'sadness': 235, 'surprise': 150, 'trust': 494}

YT sentiment: {'anger': 55, 'anticipation': 461, 'disgust': 92, 'fear': 205, 'joy': 169, 'negative': 122, 'positive': 636, 'sadness': 93, 'surprise': 88, 'trust': 417}

Is PA's Postive Sentiment > YT's Postive Sentiment? True

Mann Whitney-u Test:
MannWhitney U Value: 62.0
MannWhitney rho Value: 0.3846730627355087


# Removing Special characters
This method removes the special characters, articles, un-necessary words before steming and lemmatization

In [15]:
def remove_special_characters_from_lines(lines):
    total_clean_word_list = list()
    line_wise_word_list = list()
    for sentence in lines:
        tokens = nltk.word_tokenize(sentence)
        if 'P' in tokens:
            tokens.remove('P')
        tokens = list(filter(lambda x: x, map(lambda x: re.sub(r'[^A-Za-z0-9]+', '', x), tokens)))
        # remove articles
        tokens = [token for token in tokens if token.lower() not in ['a', 'an', 'the']]
        tagged = nltk.pos_tag(tokens)

        # lemmatize and stem the words
        stemmer = nltk.stem.PorterStemmer()
        lemmatizer = WordNetLemmatizer()

        clean_word_list = list()
        for x, y in tagged:
            x = x.lower()
            if y in adjectives:
                clean_word_list.append(lemmatizer.lemmatize(x, pos='a'))
            elif y in verbs:
                clean_word_list.append(lemmatizer.lemmatize(x, pos='v'))
            elif y in nouns:
                clean_word_list.append(stemmer.stem(x))
            else:
                clean_word_list.append(x)
        total_clean_word_list.extend(clean_word_list)
        line_wise_word_list.append(clean_word_list)
    return total_clean_word_list, line_wise_word_list

tokenize, remove special characters, stem and lemmatize

In [16]:
pa_cleaned_up, _ = remove_special_characters_from_lines(pa_group)
yt_cleaned_up, _ = remove_special_characters_from_lines(yt_group)

# Lexical Diversity Analyser

In [17]:
def lexical_diversity_analyser(words_grouping):
    return len(set(words_grouping)) / len(words_grouping)

In [18]:
# calculate the lexical diversity
pa_ld = lexical_diversity_analyser(pa_cleaned_up)
yt_ld = lexical_diversity_analyser(yt_cleaned_up)

print("PA lexical diversity: %s" % pa_ld)
print("YT lexical diversity: %s" % yt_ld)

print("\nPA>YT? %s" % (pa_ld > yt_ld))

PA lexical diversity: 0.08002443494196701
YT lexical diversity: 0.08059488245226652

PA>YT? False


# Personal Pronoun Analysis
This method counts the number of personal pronouns in two separate corpuses for analysis. The pronouns are ```'i' 'me' 'our' 'my' 'we' 'us' 'you' 'your' 'she' 'her' 'he' 'him' 'his' 'they' 'them' 'their'```

In [19]:
def personal_pronoun_analysis(words_grouping):
    pronouns_to_count = WORD_LIST
    return dict((x, words_grouping.count(x)) for x in set(pronouns_to_count))

In [20]:
pa_personal_pronouns = personal_pronoun_analysis(pa_cleaned_up)
print("PA personal pronouns: %s" % pa_personal_pronouns)

yt_personal_pronouns = personal_pronoun_analysis(yt_cleaned_up)
print("\nYT personal pronouns: %s\n" % yt_personal_pronouns)

print("Count of SHE in PA > YT ? %s" % (pa_personal_pronouns.get('she') > yt_personal_pronouns.get('she')))
print("Count of HE in PA > YT ? %s" % (pa_personal_pronouns.get('he') > yt_personal_pronouns.get('he')))
print("Count of HER in PA > YT ? %s" % (pa_personal_pronouns.get('her') > yt_personal_pronouns.get('her')))
print("Count of HIS in PA > YT ? %s\n" % (pa_personal_pronouns.get('his') > yt_personal_pronouns.get('his')))


# calculate MannWhitney-U test
mw_stat, mw_p = mann_whitney_u_test(pa_personal_pronouns, yt_personal_pronouns)
print("MannWhitney U Value: %s" % mw_stat)
print("MannWhitney rho Value: %s" % mw_p)

PA personal pronouns: {'his': 59, 'our': 37, 'them': 28, 'he': 281, 'i': 766, 'us': 58, 'you': 164, 'they': 96, 'their': 14, 'me': 63, 'she': 5, 'we': 127, 'my': 44, 'her': 6, 'him': 34, 'your': 12}

YT personal pronouns: {'his': 6, 'our': 80, 'them': 29, 'he': 28, 'i': 842, 'us': 45, 'you': 130, 'they': 171, 'their': 16, 'me': 77, 'she': 2, 'we': 149, 'my': 50, 'her': 1, 'him': 3, 'your': 8}

Count of SHE in PA > YT ? True
Count of HE in PA > YT ? True
Count of HER in PA > YT ? True
Count of HIS in PA > YT ? True

Mann Whitney-u Test:
MannWhitney U Value: 145.0
MannWhitney rho Value: 0.5339542575755475


# Get the top 10 keywords from each topic
Top 10 keywords that are representative of the topic are created by ```show_topics()```

In [21]:
# Show top n keywords for each topic
def show_topics(vectorizer, lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

# Create the Document-Word matrix
The LDA topic model algorithm requires a document word matrix as the main input.

We can create one using CountVectorizer. In the below code, I have configured the CountVectorizer to consider words that has occurred at least 10 times (min_df), remove built-in english stopwords, convert all words to lowercase, and a word can contain numbers and alphabets of at least length 3 in order to be qualified as a word.

So, to create the doc-word matrix, we need to first initialise the CountVectorizer class with the required configuration and then apply fit_transform to actually create the matrix.

Sparsicity is nothing but the percentage of non-zero datapoints in the document-word matrix, that is data_vectorized.

In [22]:
def get_vectorized_data(data):
    data_words = list(sent_to_words(data))

    # Do lemmatization keeping only Noun, Adj, Verb, Adverb
    data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

    # print(data_lemmatized)

    vectorizer = CountVectorizer(
        analyzer='word',
        min_df=10,  # minimum reqd occurences of a word
        stop_words='english',  # remove stop words
        lowercase=True,  # convert all words to lowercase
        token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
    )

    data_vectorized = vectorizer.fit_transform(data_lemmatized)

    # Materialize the sparse data
    data_dense = data_vectorized.todense()

    # Compute Sparsicity = Percentage of Non-Zero cells
    print("Sparsicity: ", ((data_dense > 0).sum() / data_dense.size) * 100, "%")

    return vectorizer, data_vectorized

# GridSearch to find the best LDA model
The most important tuning parameter for LDA models is n_components (number of topics). In addition, learning_decay (which controls the learning rate) is important as well.

Besides these, other possible search params could be learning_offset (downweigh early iterations. Should be > 1) and max_iter.

The best topic model and its parameters are printed.

```Warning: process can consume a lot of time and resources.```

In [23]:
def get_optimized_lda(data):
    vectorizer, data_vectorized = get_vectorized_data(data)

    # Define Search Param
    search_params = {'n_components': [10, 15, 20, 25, 30, 35, 40], 'learning_decay': [0.1, 0.3, .5, .7, .9]}

    # Init the Model
    lda = LatentDirichletAllocation()

    # Init Grid Search Class
    model = GridSearchCV(lda, param_grid=search_params, cv=3, iid=True)

    # Do the Grid Search
    model.fit(data_vectorized)

    # Best Model
    best_lda_model = model.best_estimator_
    print(best_lda_model)
    # Model Parameters
    print("Best Model's Params: ", model.best_params_)

    # Log Likelihood Score
    print("Best Log Likelihood Score: ", model.best_score_)

    # Perplexity
    print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))
    return best_lda_model, data_vectorized, vectorizer

 # Finding dominant topic in each document
To classify a document as belonging to a particular topic, a logical approach is to see which topic has the highest contribution to that document and assign it.

In [24]:
def create_document_topic_matrix(best_lda_model, data_vectorized):
    # Create Document - Topic Matrix

    lda_output = best_lda_model.transform(data_vectorized)

    # column names
    topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_components)]

    # index names
    docnames = ["Doc" + str(i) for i in range(int(lda_output.size/best_lda_model.n_components))]
    
    # Make the pandas dataframe
    df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

    # Get dominant topic for each document
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic['dominant_topic'] = dominant_topic

    # Apply Style
    df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)

    return df_document_topics, df_document_topic, topicnames


# Review topics distribution across documents

In [25]:
def get_topic_distribution_over_documents(df_document_topic):
    df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
    df_topic_distribution.columns = ['Topic Num', 'Num Documents']
    return df_topic_distribution

# Visualize the LDA model with pyLDAvis
The pyLDAvis offers the best visualization to view the topics-keywords distribution.

A good topic model will have non-overlapping, fairly big sized blobs for each topic.

In [26]:
def visualize_lda_model(best_lda_model, data_vectorized, vectorizer):
    pyLDAvis.enable_notebook()
    panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')
    return panel


# Get Topic’s keywords
The weights of each keyword in each topic is contained in lda_model.components_ as a 2d array. The names of the keywords itself can be obtained from vectorizer object using get_feature_names().

Let’s use this info to construct a weight matrix for all keywords in each topic.

In [27]:
def get_topic_keywords(best_lda_model, vectorizer, topicnames):
    # Topic-Keyword Matrix
    df_topic_keywords = pd.DataFrame(best_lda_model.components_)

    # Assign Column and Index
    df_topic_keywords.columns = vectorizer.get_feature_names()
    df_topic_keywords.index = topicnames
    return df_topic_keywords

# Get the top 10 keywords of each topic

In [28]:
def get_top_ten_keywords_in_topics(best_lda_model, vectorizer):
    topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=10)

    # Topic - Keywords Dataframe
    df_topic_keywords = pd.DataFrame(topic_keywords)
    df_topic_keywords.columns = ['Word ' + str(i) for i in range(df_topic_keywords.shape[1])]
    df_topic_keywords.index = ['Topic ' + str(i) for i in range(df_topic_keywords.shape[0])]
    return df_topic_keywords

# Build LDA model with sklearn
Latent Dirichlet Allocation (LDA) is initialised and fit_transform() is called to build the LDA model.

```Note: This is not used here but we can use the optimized params from GridSearch Algorithm give above to initialized and fit the model.```

In [29]:
def analyser(data):
    _, data_vectorized = get_vectorized_data(data)
    # Build LDA Model
    lda_model = LatentDirichletAllocation(
        n_components=20,  # Number of topics
        max_iter=10,  # Max learning iterations
        learning_method='online',
        random_state=100,  # Random state
        batch_size=128,  # n docs in each learning iter
        evaluate_every=-1,  # compute perplexity every n iters, default: Don't
        n_jobs=-1,  # Use all available CPUs
    )
    lda_output = lda_model.fit_transform(data_vectorized)

    print(lda_output)

    # Log Likelyhood: Higher the better
    print("Log Likelihood: ", lda_model.score(data_vectorized))

    # Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
    print("Perplexity: ", lda_model.perplexity(data_vectorized))

    # See model parameters
    pprint(lda_model.get_params())

In [30]:
# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)


def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

Get the optimized LDA parameters for each corpuses

In [31]:
print("Optimal LDA Parameters for PA corpus:")
best_lda_model_pa, data_vectorized_pa, vectorizer_pa = get_optimized_lda(pa_group)

Optimal LDA Parameters for PA corpus:
Sparsicity:  2.959816003294264 %
LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.1,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)
Best Model's Params:  {'learning_decay': 0.1, 'n_components': 10}
Best Log Likelihood Score:  -6602.104849631188
Model Perplexity:  108.95924434536923


In [32]:
print("Optimal LDA Parameters for YT corpus:")
best_lda_model_yt, data_vectorized_yt, vectorizer_yt = get_optimized_lda(yt_group)

Optimal LDA Parameters for YT corpus:
Sparsicity:  3.286978508217446 %
LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)
Best Model's Params:  {'learning_decay': 0.7, 'n_components': 10}
Best Log Likelihood Score:  -6070.366066154976
Model Perplexity:  106.35664469985527


In [33]:
print("Document-topic matrix for PA corpus:")
df_document_topics_pa, df_document_topic_pa, topicnames_pa = create_document_topic_matrix(best_lda_model_pa, data_vectorized_pa)
df_document_topics_pa

Document-topic matrix for PA corpus:


,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,dominant_topic
Doc0,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.55,0.05,0.05,7
Doc1,0.01,0.47,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.43,1
Doc2,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0
Doc3,0.03,0.03,0.03,0.03,0.37,0.03,0.03,0.37,0.03,0.03,4
Doc4,0.02,0.02,0.02,0.22,0.02,0.22,0.02,0.02,0.02,0.42,9
Doc5,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.55,0.05,8
Doc6,0.01,0.26,0.64,0.01,0.01,0.01,0.01,0.01,0.01,0.01,2
Doc7,0.01,0.24,0.01,0.01,0.01,0.42,0.01,0.01,0.24,0.01,5
Doc8,0.03,0.03,0.77,0.03,0.03,0.03,0.03,0.03,0.03,0.03,2
Doc9,0.29,0,0.29,0,0,0.14,0,0,0.26,0,0


In [34]:
print("Document-topic matrix for YT corpus:")
df_document_topics_yt, df_document_topic_yt, topicnames_yt = create_document_topic_matrix(best_lda_model_yt, data_vectorized_yt)
df_document_topics_yt

Document-topic matrix for YT corpus:


,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,dominant_topic
Doc0,0.02,0.02,0.02,0.45,0.39,0.02,0.02,0.02,0.02,0.02,3
Doc1,0.01,0.01,0.01,0.75,0.17,0.01,0.01,0.01,0.01,0.01,3
Doc2,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0
Doc3,0.01,0.01,0.46,0.48,0.01,0.01,0.01,0.01,0.01,0.01,3
Doc4,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0
Doc5,0.03,0.03,0.7,0.03,0.03,0.03,0.03,0.03,0.03,0.03,2
Doc6,0.03,0.03,0.52,0.03,0.03,0.03,0.03,0.03,0.03,0.28,2
Doc7,0.28,0.01,0.01,0.01,0.27,0.29,0.01,0.01,0.12,0.01,5
Doc8,0.82,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0
Doc9,0.05,0.05,0.05,0.05,0.05,0.05,0.55,0.05,0.05,0.05,6


In [35]:
print("Topics distribution over documents in PA corpus:")
df_topic_distribution_pa = get_topic_distribution_over_documents(df_document_topic_pa)
df_topic_distribution_pa

Topics distribution over documents in PA corpus:


,Topic Num,Num Documents
0,0,316
1,4,78
2,1,78
3,5,76
4,7,73
5,9,59
6,8,58
7,3,56
8,6,54
9,2,49


In [36]:
print("Topics distribution over documents in YT corpus:")
df_topic_distribution_yt = get_topic_distribution_over_documents(df_document_topic_yt)
df_topic_distribution_yt

Topics distribution over documents in YT corpus:


,Topic Num,Num Documents
0,0,259
1,2,87
2,5,70
3,3,69
4,4,64
5,6,53
6,8,52
7,7,52
8,9,48
9,1,37


In [37]:
print("Visualization of topics and most influencial words in the topics in PA corpus:")
pa_panel = visualize_lda_model(best_lda_model_pa, data_vectorized_pa, vectorizer_pa)
pa_panel

Visualization of topics and most influencial words in the topics in PA corpus:


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
0      -46.227982 -121.545151       1        1  13.703024
5       14.550008  -62.939949       2        1  12.105677
2       11.716529   21.445324       3        1  11.939773
8       85.814674  -91.821899       4        1  11.617404
1     -120.329887   -8.285069       5        1  11.614430
9      -49.062721  -37.161987       6        1  10.911418
4      -58.604309   47.300724       7        1   8.493124
7       24.089130 -147.406601       8        1   8.350740
6       80.199028   -8.946818       9        1   6.210733
3     -114.711777  -91.159492      10        1   5.053677, topic_info=    Category        Freq         Term       Total  loglift  logprob
95   Default  104.000000        think  104.000000  30.0000  30.0000
91   Default   74.000000       target   74.000000  29.0000  29.0000
45   Default   84.000000         look   84.000000  28.0000  28.0000
34   Default  126.000000   interested  126.000000  27.0000  27.0000
110  Default   42.000000      youtube   42.000000  26.0000  26.0000
39   Default  100.000000         just  100.000000  25.0000  25.0000
62   Default   91.000000      persona   91.000000  24.0000  24.0000
4    Default   61.000000     audience   61.000000  23.0000  23.0000
41   Default   92.000000         know   92.000000  22.0000  22.0000
28   Default   66.000000        group   66.000000  21.0000  21.0000
83   Default   74.000000       social   74.000000  20.0000  20.0000
51   Default   35.000000         mean   35.000000  19.0000  19.0000
70   Default   39.000000       really   39.000000  18.0000  18.0000
108  Default   29.000000        write   29.000000  17.0000  17.0000
56   Default   75.000000         news   75.000000  16.0000  16.0000
26   Default   36.000000         good   36.000000  15.0000  15.0000
53   Default   35.000000       minute   35.000000  14.0000  14.0000
38   Default   39.000000       jordan   39.000000  13.0000  13.0000
43   Default   59.000000         like   59.000000  12.0000  12.0000
20   Default   65.000000          don   65.000000  11.0000  11.0000
74   Default   28.000000     remember   28.000000  10.0000  10.0000
106  Default   41.000000        watch   41.000000   9.0000   9.0000
78   Default   58.000000          say   58.000000   8.0000   8.0000
52   Default   49.000000       medium   49.000000   7.0000   7.0000
64   Default   47.000000      politic   47.000000   6.0000   6.0000
93   Default   20.000000        thank   20.000000   5.0000   5.0000
90   Default   31.000000        syria   31.000000   4.0000   4.0000
96   Default   38.000000         time   38.000000   3.0000   3.0000
109  Default   28.000000         year   28.000000   2.0000   2.0000
94   Default   38.000000        thing   38.000000   1.0000   1.0000
..       ...         ...          ...         ...      ...      ...
73   Topic10    2.154549     relation   17.417635   0.8952  -4.4372
12   Topic10    1.469707      compare   14.576555   0.6907  -4.8197
89   Topic10    1.415866         sure   18.032242   0.4406  -4.8570
103  Topic10    4.955931        video   64.508189   0.4188  -3.6042
23   Topic10    2.202029     election   31.386240   0.3281  -4.4154
76   Topic10    2.446431        right   37.756554   0.2485  -4.3101
98   Topic10    2.544590        topic   45.932836   0.0918  -4.2708
39   Topic10    5.073273         just  100.531379  -0.0014  -3.5808
107  Topic10    1.280386         work   34.804545  -0.3175  -4.9576
69   Topic10    1.253095         read   35.759784  -0.3662  -4.9791
65   Topic10    0.098768     probably   10.458312  -1.6773  -7.5197
58   Topic10    0.098768         okay   10.617336  -1.6924  -7.5197
24   Topic10    0.098772        email   11.644890  -1.7848  -7.5197
85   Topic10    0.098771     specific   11.789786  -1.7971  -7.5197
100  Topic10    0.098786   understand   12.366127  -1.8447  -7.5195
16   Topic10    0.098772         didn   12.724248  -1.8734  -7.51

In [38]:
print("Visualization of topics and most influencial words in the topics in YT corpus:")
yt_panel = visualize_lda_model(best_lda_model_yt, data_vectorized_yt, vectorizer_yt)
yt_panel

Visualization of topics and most influencial words in the topics in YT corpus:


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
3       64.110062   -9.287521       1        1  14.275594
9       16.887762  -85.433990       2        1  12.803862
8      135.351013 -106.181511       3        1  12.600456
5      140.446259  -56.625397       4        1  11.519809
4       93.222359 -132.770065       5        1  10.387494
0       54.279297 -110.350716       6        1   9.231685
2      103.057365  -31.701908       7        1   8.539997
6       96.159256  -81.874435       8        1   7.383269
7       21.981749  -35.877430       9        1   7.327554
1       61.176800  -60.180786      10        1   5.930280, topic_info=    Category        Freq         Term       Total  loglift  logprob
43   Default  120.000000         know  120.000000  30.0000  30.0000
21   Default   90.000000          don   90.000000  29.0000  29.0000
3    Default   69.000000     audience   69.000000  28.0000  28.0000
97   Default  132.000000        watch  132.000000  27.0000  27.0000
85   Default   89.000000         time   89.000000  26.0000  26.0000
93   Default  118.000000        video  118.000000  25.0000  25.0000
51   Default   84.000000         male   84.000000  24.0000  24.0000
39   Default  127.000000       jordan  127.000000  23.0000  23.0000
41   Default  101.000000         just  101.000000  22.0000  22.0000
37   Default   52.000000   interested   52.000000  21.0000  21.0000
68   Default   54.000000          say   54.000000  20.0000  20.0000
48   Default  112.000000         look  112.000000  19.0000  19.0000
13   Default   46.000000      content   46.000000  18.0000  18.0000
61   Default   40.000000       people   40.000000  17.0000  17.0000
94   Default   79.000000         view   79.000000  16.0000  16.0000
18   Default   27.000000         didn   27.000000  15.0000  15.0000
66   Default   20.000000     remember   20.000000  14.0000  14.0000
28   Default   22.000000         good   22.000000  13.0000  13.0000
102  Default   36.000000         year   36.000000  12.0000  12.0000
84   Default   59.000000        think   59.000000  11.0000  11.0000
83   Default   37.000000        thing   37.000000  10.0000  10.0000
103  Default   60.000000      youtube   60.000000   9.0000   9.0000
101  Default   21.000000        write   21.000000   8.0000   8.0000
8    Default   27.000000        click   27.000000   7.0000   7.0000
92   Default   27.000000          use   27.000000   6.0000   6.0000
52   Default   33.000000          man   33.000000   5.0000   5.0000
79   Default   47.000000       target   47.000000   4.0000   4.0000
75   Default   22.000000        state   22.000000   3.0000   3.0000
10   Default   18.000000      comment   18.000000   2.0000   2.0000
30   Default   52.000000        group   52.000000   1.0000   1.0000
..       ...         ...          ...         ...      ...      ...
40   Topic10    9.053814    jordanian   15.151338   2.3102  -3.0818
15   Topic10    8.967546        datum   16.943656   2.1888  -3.0914
64   Topic10   12.919238       really   24.776581   2.1739  -2.7263
37   Topic10   25.665054   interested   52.050951   2.1180  -2.0399
13   Topic10   20.448070      content   46.065662   2.0129  -2.2671
80   Topic10    5.930826         tell   17.685509   1.7325  -3.5049
35   Topic10    4.150426    important   13.537972   1.6428  -3.8618
77   Topic10    2.980487      suppose   10.776684   1.5398  -4.1929
17   Topic10    8.677116  demographic   36.929212   1.3768  -3.1243
46   Topic10    4.237982         live   18.864129   1.3319  -3.8409
57   Topic10    3.923576         news   19.719669   1.2105  -3.9180
20   Topic10    2.361945        doesn   12.557886   1.1542  -4.4255
33   Topic10    2.159994         help   11.766755   1.1299  -4.5149
98   Topic10    2.159980          way   11.952814   1.1142  -4.5149
42   Topic10    3.017736         kind   19.225954   0.9733  -4.1805
21   Topic10   11.533255          don   90.151129   0.7688  -2.83

In [39]:
print("Weights of each keyword in each topic in PA corpus:")
df_topic_keywords_pa = get_topic_keywords(best_lda_model_pa, vectorizer_pa, topicnames_pa)
df_topic_keywords_pa.head()

Weights of each keyword in each topic in PA corpus:


,actually,age,american,analytic,audience,average,base,big,click,college,...,united,use,video,view,want,watch,work,write,year,youtube
Topic0,0.100004,0.100008,0.100016,0.100000,0.100007,9.592538,0.100007,0.100005,0.100000,5.087821,...,0.100011,0.100007,5.903620,20.040123,0.100010,0.100026,13.787923,0.100003,0.100012,0.100000
Topic1,0.100004,21.121461,0.100002,0.100004,51.437529,0.100013,0.100006,3.958667,0.100000,0.100002,...,0.100011,0.100007,5.053598,0.100021,0.100013,0.125023,0.100010,0.100003,0.100000,0.100012
Topic2,0.100010,0.100005,10.417634,0.100000,0.100004,2.121460,7.984066,0.100011,0.100004,0.100008,...,0.100009,0.100002,8.325076,17.216403,0.100010,33.639618,0.268536,0.100004,0.100005,0.100006
Topic3,0.100005,0.100002,0.100014,15.099913,0.100005,0.100002,0.100001,0.100000,11.765044,0.100005,...,0.100010,2.470046,5.017742,0.100015,0.100001,0.100007,1.296355,0.100006,0.100005,41.373037
Topic4,0.100022,0.100005,0.100002,0.100000,0.100013,0.100010,0.100002,0.100000,0.100009,0.100004,...,0.100000,15.729909,0.140123,0.100009,7.994741,0.100015,0.100026,0.100004,0.100005,0.100005


In [40]:
print("Weights of each keyword in each topic in YT corpus:")
df_topic_keywords_yt = get_topic_keywords(best_lda_model_yt, vectorizer_yt, topicnames_yt)
df_topic_keywords_yt.head()

Weights of each keyword in each topic in YT corpus:


,actually,age,analytic,audience,average,big,bit,channel,click,come,...,view,viewer,want,watch,way,woman,work,write,year,youtube
Topic0,0.100028,2.430304,0.100005,0.141948,0.100005,0.100006,0.100013,0.100010,0.178826,7.735936,...,0.104253,0.100001,3.392822,0.100015,0.100000,0.100007,0.100041,0.100003,2.932587,0.100012
Topic1,2.698557,0.100009,0.100024,1.691851,0.100000,0.100007,0.100009,0.100003,0.100004,0.100039,...,0.100007,0.100000,0.100030,7.433598,2.100046,0.100009,0.100009,0.100004,0.100001,0.100002
Topic2,0.100002,3.663489,0.100015,0.100009,0.100005,0.100007,0.100013,0.100019,0.100004,0.100008,...,0.100008,0.100000,11.190500,0.104609,0.100003,0.100001,9.094809,23.099965,27.017016,5.641429
Topic3,0.100008,21.105734,3.020401,0.100019,5.720724,0.100002,0.100010,0.100010,0.100005,0.100007,...,4.834776,0.100026,0.218837,10.398962,1.099912,1.616623,0.100000,0.100000,0.100019,0.103831
Topic4,0.100006,9.301983,14.176691,61.050575,0.100006,6.099961,0.100002,12.099988,0.100008,0.100006,...,0.100015,0.100010,21.435787,0.105057,0.100001,1.847868,0.112506,0.100007,6.058396,28.372815


In [41]:
print('Top 10 keywords in each topic in PA ccorpus:')
df_topic_keywords_pa = get_top_ten_keywords_in_topics(best_lda_model_pa, vectorizer_pa)
df_topic_keywords_pa

Top 10 keywords in each topic in PA ccorpus:


,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9
Topic 0,like,news,read,iraq,innovation,research,refugee,politic,view,election
Topic 1,target,group,audience,jordan,size,age,reach,number,just,current
Topic 2,interested,watch,syria,social,view,research,innovation,thing,know,maybe
Topic 3,youtube,mean,thank,analytic,easy,click,difficult,lot,just,video
Topic 4,look,know,don,remember,stuff,thing,use,title,comment,come
Topic 5,think,say,content,actually,want,just,need,video,right,story
Topic 6,write,minute,information,time,guy,video,right,doesn,know,important
Topic 7,just,really,good,people,interesting,person,different,persona,say,like
Topic 8,persona,social,medium,interested,politic,news,general,big,issue,audience
Topic 9,year,state,look,united,male,old,age,phil,persona,live


In [42]:
print('Top 10 keywords in each topic in YT ccorpus:')
df_topic_keywords_yt = get_top_ten_keywords_in_topics(best_lda_model_yt, vectorizer_yt)
df_topic_keywords_yt

Top 10 keywords in each topic in YT ccorpus:


,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9
Topic 0,know,don,thing,need,time,interesting,come,mean,guess,look
Topic 1,interested,content,remember,really,don,story,jordanian,datum,demographic,watch
Topic 2,just,look,year,write,thank,email,information,segment,want,male
Topic 3,male,jordan,look,try,age,let,demographic,gender,female,percentage
Topic 4,audience,youtube,man,target,state,want,united,try,analytic,channel
Topic 5,watch,time,just,number,minute,actually,long,video,jordan,thing
Topic 6,say,think,comment,just,easy,day,viewer,age,persona,way
Topic 7,people,didn,good,use,maybe,interested,know,happen,make,lot
Topic 8,video,jordan,group,view,come,age,watch,second,content,topic
Topic 9,view,watch,click,video,look,just,time,youtube,total,duration


# Write to each corpus file 
write the classified text to each corpus files

In [43]:
def write_to_corpus_file(data, _type=PA_CORPUS_TEXT):
    text_file_path = os.path.join(OUTPUT_PATH, _type)
    with open(text_file_path, 'w') as outfile:
        for line in data:
            outfile.write("%s\n" % line)

In [44]:
# write to pa corpus text file
write_to_corpus_file(pa_group, PA_CORPUS_TEXT)

# write to yt corpus text file
write_to_corpus_file(yt_group, YT_CORPUS_TEXT)